# Lab 3 — clustering

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sklearn.cluster as cl
import random as rand

%matplotlib inline
plt.style.use("ggplot")

In [8]:
#centroids = []
# Load tag2coord
#for k in range(1,5):
    #centroids[k] = cl.k_means(tag2coord, k)

In [93]:
a = set()
a.add('a')
a.add('b')
b = set()
b.add('c')
b.add('d')
a = a.union(b)
print(a)
k_medioids(a, distance, 1)
#m = [set()]*10
#print(m)

{'a', 'b', 'c', 'd'}


NameError: name 'y' is not defined

In [6]:
def distance(A, B):
    jaccard_index = len(A.intersection(B))/len(A.union(B)) 
    return 1 - jaccard_index

In [92]:
def k_medioids(X, d, k):
    m = [set()]*len(X)
    for i in range(0,k):
        m[i] = set(rand.sample(X, 1))
    convergence = 0
    C = [set()]*k
    # use np.isclose() for convergence, when do we know if it converges?
    while not convergence:
        # Initialise Ci to empty set
        for i in range(0,k):
            C[i] = set()
        ind = 0
        for x in X:
            ind = np.argmin(d(m[ind], set(x)), ind)
            C[ind] = set(C[ind].union(set(x)))
        for i in range(0,k):
            # Faux, comment faire somme interne a argmin?
            m[i] = np.argmin(sum(d(set(C[i]), set(y))), C[i])

In [30]:
rand.randint(0,1)

1